In [3]:
import sys
sys.path.append('../')
from queries import *
df = query_sparql('../queries/sparql_query.txt')

ModuleNotFoundError: No module named 'SPARQLWrapper'

In [ ]:
#df.to_csv('../data/countries.csv')
import pycountry

df = pd.read_csv('../data/countries.csv')

countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_3

codes = [countries.get(country, 'Unknown code') for country in df.country_name]
df['ISO'] = codes
df.head()

,Unnamed: 0,country,capital,country_name,capital_name,country_flag,country_abstract,ISO
0,0,http://dbpedia.org/resource/Afghanistan,http://dbpedia.org/resource/Kabul,Afghanistan,Kabul,http://commons.wikimedia.org/wiki/Special:File...,"Afghanistan (/æfˈɡænɪstæn, æfˈɡɑːnɪstɑːn/), of...",AFG
1,1,http://dbpedia.org/resource/Albania,http://dbpedia.org/resource/Tirana,Albania,Tirana,http://commons.wikimedia.org/wiki/Special:File...,"Albania (/ælˈbeɪniə, ɔːl-/ a(w)l-BAY-nee-ə; Al...",ALB
2,2,http://dbpedia.org/resource/Algeria,http://dbpedia.org/resource/Algiers,Algeria,Algiers,http://commons.wikimedia.org/wiki/Special:File...,"Algeria, officially the People's Democratic Re...",DZA
3,3,http://dbpedia.org/resource/Andorra,http://dbpedia.org/resource/Andorra_la_Vella,Andorra,Andorra la Vella,http://commons.wikimedia.org/wiki/Special:File...,"Andorra, officially the Principality of Andorr...",AND
4,4,http://dbpedia.org/resource/Angola,http://dbpedia.org/resource/Luanda,Angola,Luanda,http://commons.wikimedia.org/wiki/Special:File...,"Angola (/ænˈɡoʊlə/; Portuguese: [ɐ̃ˈɡɔlɐ]), of...",AGO


In [2]:
sys.path.append('../')
from queries import *


df_countries = query_sparql('../queries/sparql_query.txt')
df_borders = query_wikidata('../queries/sparql_wikidata_query.txt')

# Adding ISO Code
countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_3
codes = [countries.get(country, 'Unknown code') for country in df_countries.country_name]
df_countries['ISO'] = codes

# Preparing the borders
df_borders_land = df_borders[df_borders['isLandBorder']=='true']
df_borders_tbm = df_borders_land[['country1Label','country2Label']].groupby('country1Label').agg({'country2Label':list}).reset_index()

# Preparing the borders
countries = pd.merge(df_countries, df_borders_tbm, right_on = 'country1Label', left_on='country_name', how= 'left')
countries

countries.to_csv('../data/countries.csv')

In [3]:
import pandas as pd 
import sys
sys.path.append('../')
from utils.map import *

#game_data = {'Ger': ['Germany','DEU', 'True'], 'Fr': ['France','FRA','False']}
#game_data =  pd.DataFrame.from_dict(game_data, orient='index',columns = ['countries','ISO','played'])

game_data = countries
game_data['played'] = game_data.apply(lambda x: True if x.country2Label != [] else False, axis=1)
game_data
show_map_game(game_data)